<a href="https://colab.research.google.com/github/mateusribeirocampos/diollm/blob/main/YOLOv3_Dec_seg_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
!git clone https://github.com/AlexeyAB/darknet.git
%cd darknet
!make

Cloning into 'darknet'...
remote: Enumerating objects: 15873, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15873 (delta 3), reused 0 (delta 0), pack-reused 15863 (from 3)
Receiving objects: 100% (15873/15873), 14.47 MiB | 13.00 MiB/s, done.
Resolving deltas: 100% (10686/10686), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ []8;;https://gcc.gnu.org/onlinedocs/gcc/W

In [3]:
!sudo apt-get install tree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 47.9 kB of archives.
After this operation, 116 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tree amd64 2.0.2-1 [47.9 kB]
Fetched 47.9 kB in 0s (127 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package tree.
(Reading database ... 124926 files and directories currently install

In [5]:
!mkdir -p /content/coco/images
!mkdir -p /content/coco/annotations

In [16]:
# Baixar imagens de treino (18GB)
#!wget -c http://images.cocodataset.org/zips/train2017.zip -P /content/coco/images
# Baixar anotações de treino/validação (241MB)
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P /content/coco/annotations

--2025-02-15 21:17:39--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.170.251, 3.5.25.130, 3.5.28.247, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.170.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘/content/coco/annotations/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  25.5MB/s    in 9.5s    

2025-02-15 21:17:49 (25.4 MB/s) - ‘/content/coco/annotations/annotations_trainval2017.zip’ saved [252907541/252907541]



In [17]:
# Descompactar imagens
#!unzip /content/coco/images/train2017.zip -d /content/coco/images
#!rm /content/coco/images/train2017.zip  # Remover o ZIP para economizar espaço

# Descompactar anotações
!unzip /content/coco/annotations/annotations_trainval2017.zip -d /content/coco/annotations

!rm /content/coco/annotations/annotations_trainval2017.zip

Archive:  /content/coco/annotations/annotations_trainval2017.zip
replace /content/coco/annotations/annotations/instances_train2017.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Baixar val2017 (1GB)
!wget -c http://images.cocodataset.org/zips/val2017.zip -P /content/coco/images
!unzip /content/coco/images/val2017.zip -d /content/coco/images
!rm /content/coco/images/val2017.zip

# Baixar test2017
!wget -c http://images.cocodataset.org/zips/test2017.zip -P /content/coco/images
!unzip /content/coco/images/test2017.zip -d /content/coco/images
!rm /content/coco/images/test2017.zip

In [21]:
# Criar diretório para anotações
!mkdir -p /content/coco/annotations

# Baixar anotações (train + val)
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P /content/coco/annotations

# Descompactar e remover ZIP
!unzip /content/coco/annotations/annotations_trainval2017.zip -d /content/coco/annotations
!rm /content/coco/annotations/annotations_trainval2017.zip

--2025-02-15 21:22:19--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.25.231, 16.182.38.217, 3.5.28.98, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.25.231|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘/content/coco/annotations/annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  61.2MB/s    in 4.4s    

2025-02-15 21:22:24 (54.5 MB/s) - ‘/content/coco/annotations/annotations_trainval2017.zip’ saved [252907541/252907541]

Archive:  /content/coco/annotations/annotations_trainval2017.zip
  inflating: /content/coco/annotations/annotations/instances_train2017.json  
  inflating: /content/coco/annotations/annotations/instances_val2017.json  
  inflating: /content/coco/annotations/annotations/captions_train2017.json  
  inflating: /content/coco/annotations/annotations/cap

In [29]:
!ls /content/coco/annotations/

captions_train2017.json  instances_train2017.json  person_keypoints_train2017.json
captions_val2017.json	 instances_val2017.json    person_keypoints_val2017.json


In [27]:
# Comandos para corrigir a estrutura de pastas
!mv /content/coco/annotations/annotations/*.json /content/coco/annotations/
!rm -r /content/coco/annotations/annotations  # Remova a subpasta vazia

In [31]:
import os
import shutil
from pycocotools.coco import COCO

def process_coco_split(splits: list, categories: list, max_samples_per_split: int = None):
    """
    Processa splits do COCO (train, val, test), filtrando por categorias e convertendo para YOLO.
    """
    # Criar diretório principal
    os.makedirs('/content/coco/filtered/', exist_ok=True)

    # Criar arquivo de classes
    with open('/content/coco/filtered/obj.names', 'w') as f:
        for cat in categories:
            f.write(cat + '\n')

    for split in splits:
        print(f"\nProcessando split: {split}")

        # Caminhos das imagens
        img_source_dir = f'/content/coco/images/{split}2017'
        img_dest_dir = f'/content/coco/filtered/{split}/images'
        os.makedirs(img_dest_dir, exist_ok=True)

        # Processar apenas train/val (test não tem anotações)
        if split != 'test':
            # Carregar anotações
            coco = COCO(f'/content/coco/annotations/instances_{split}2017.json')
            cat_ids = coco.getCatIds(catNms=categories)
            img_ids = coco.getImgIds(catIds=cat_ids)

            # Limitar amostras se especificado
            if max_samples_per_split:
                img_ids = img_ids[:max_samples_per_split]

            images = coco.loadImgs(img_ids)

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

            # Criar diretório de labels
            ann_dest_dir = f'/content/coco/filtered/{split}/labels'
            os.makedirs(ann_dest_dir, exist_ok=True)

            # Converter anotações para YOLO
            for img in images:
                ann_ids = coco.getAnnIds(imgIds=img['id'], catIds=cat_ids)
                annotations = coco.loadAnns(ann_ids)

                label_path = os.path.join(ann_dest_dir, img['file_name'].replace('.jpg', '.txt'))
                with open(label_path, 'w') as f:
                    for ann in annotations:
                        x, y, w, h = ann['bbox']
                        img_w, img_h = img['width'], img['height']

                        cx = (x + w/2) / img_w
                        cy = (y + h/2) / img_h
                        nw = w / img_w
                        nh = h / img_h

                        class_id = categories.index(coco.loadCats(ann['category_id'])[0]['name'])
                        f.write(f"{class_id} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}\n")

        # Processar test (sem anotações)
        else:
            # Listar todas as imagens do diretório test2017
            images = [{'file_name': f} for f in os.listdir(img_source_dir)]
            if max_samples_per_split:
                images = images[:max_samples_per_split]

            # Copiar imagens
            for img in images:
                src = os.path.join(img_source_dir, img['file_name'])
                dst = os.path.join(img_dest_dir, img['file_name'])
                shutil.copy(src, dst)

# Uso
process_coco_split(
    splits=['train', 'val', 'test'],
    categories=['boat', 'person'],
    max_samples_per_split=100
)


Processando split: train
loading annotations into memory...
Done (t=16.93s)
creating index...
index created!

Processando split: val
loading annotations into memory...
Done (t=0.54s)
creating index...
index created!

Processando split: test


In [32]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2025-02-15 21:34:24--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  34.0MB/s    in 7.2s    

2025-02-15 21:34:32 (33.0 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [34]:
!ls

3rdparty	darknet			Dockerfile.cpu	       Makefile       scripts
backup		DarknetConfig.cmake.in	Dockerfile.gpu	       net_cam_v3.sh  src
build		darknet_images.py	image_yolov3.sh        net_cam_v4.sh  vcpkg.json
build.ps1	darknet.py		image_yolov4.sh        obj	      video_yolov3.sh
cfg		darknet_video.py	include		       package.xml    video_yolov4.sh
cmake		data			json_mjpeg_streams.sh  README.md      yolov3.weights
CMakeLists.txt	docker-compose.yml	LICENSE		       results
